- model:模型
- mode:模式
- module:模块

In [2]:
"""
1、agent有哪些技术要点
    理解AI Agent:
        理解1：利用人工智能技术执行特定任务
        理解2：它被设计为具有独立思考和行动能力的AI程序。
            你只需要提供一个目标，比如写一个游戏、开发一个网页，他就会根据环境的反应和独白的形式生成一个任务序列开始工作。
            就好像是人工智能可以自我提示反馈，不断发展和适应，以尽可能最好的方式来实现你给出的目标。
        理解3：AI Agent（人工智能代理）是一个自主的计算机程序，能够感知其环境并采取行动以实现特定目标
        理解4*:Agent 是连接大型语言模型（LLM）与工具和长期记忆的核心。Agent 还提供了摄取各种类型文档的能力，如 PDF、Txt、Markdown、Json 等。以下是其功能列表。
2、多轮对话,query分析(是做ner,还是什么)
    a.多轮对话中遇到的问题：后续问题缺失主题，在RAG检索知识会出现偏差，解决方法：拼接历史query
    b.拼接历史query的方法也会引出其他问题(如切换话题的情况)，综合解决大方案：构建多轮信息处理模块(单独部署该模块的LLM)
    
3、dropout
4.大模型微调中遇到的一些问题，必须要遇到问题，如推理产生幻觉的原因以及解决方法
    a.大模型训练的基本情况：
        context：4张v100 32G的gup,训练6w数据，模型为13B baichuan2,
                batch_size=4 , epoch=4,  
        需要训练17个小时
5.大模型训练
    1、pt，预训练(增量训练)
    2、sft,微调
    3、RLHF,基于人类反馈的强化学习
        a.奖励模型+ppo强化学习
        b.DPO(偏好优化)
    a.大模型微调中，最占现存的是哪些部分，优化器和计算梯度占内存情况，比如整体上的百分比
    b.并行训练框架：deepspeed,megatron-LM,colossal-ai
      大模型微调框架: LLaMA-Factory、阿里的swift，书生xtuner、Firefly等
    c.torch的使用的熟练程度，Dataset怎么写需要改写那些方法，
    d.强化学习有没有关注看没看过一些最近的论文，
    e.在大模型微调的时候针对微调效果问题有没有做一些深入研究和迭代微调
    f.训练加速框架DeepSpeed的了解
6.RAG
    向量数据库的选择：Milvus,为什么这样选择


    
"""

'\n1、agent有哪些技术要点\n2、多轮对话，query分析(是做ner,还是)\n'

# TOT(思维树)
ToT采用了分层结构来引导思考过程,  
- 分层结构：该技术使用类似树的结构，其中每个节点表示一种思想或想法。这使得可以分支到多个方向，提供了广泛的解决方案。  
- 动态评估：在每个节点上，LLM评估思路的有效性，并决定是否继续或探索其他分支。  
聚焦响应：通过引导LLM进行结构化的思考过程，ToT确保生成的回复不仅相关而且在上下文上富有内容。  
- 启发式搜索:这些算法指导LLM在树中遍历，帮助它评估每个想法或节点的有效性。它们应用一组规则或启发式来确定哪些分支值得进一步探索，哪些应该被剪枝。  它使用启发式函数来指导搜索过程。

- TOT工作流程：在有限步骤下，基于当前思维链状态链生成新的思维(generate_thoughts)作为节点子状态，然后对各带有新子状态的思维链进行评测打分(evaluate_states)，选出top优链(即思维树的多条最佳路线)，最后对选出的优链进行最后的答案生成。

# Dropout
原理与机制
Dropout 是一种在训练神经网络时防止过拟合的技术，由Hinton等人在2012年提出。它的核心思想是在每次训练过程中随机“丢弃”一部分神经元，使其暂时从网络中移除，从而迫使网络不得不在多个不同的子网络上进行训练。这可以有效地防止模型对训练数据的过拟合。

过拟合的解决
过拟合是指模型在训练数据上表现很好，但在未见过的测试数据上表现较差。Dropout通过以下方式解决过拟合问题：

模型泛化能力增强：由于每次训练时随机丢弃神经元，网络的不同部分在不同的训练过程中都会被激活。这样，模型不能依赖某些特定的神经元，而是必须学会在不同的组合中表现良好。这种训练方式可以提高模型的泛化能力。

避免共适应性（Co-adaptation）：神经元之间的共适应性会导致模型过拟合，因为某些神经元可能会“协作”去记住训练数据的某些模式。通过随机丢弃神经元，Dropout破坏了这种共适应性，使每个神经元在做决策时更独立。

## ner实体识别相关问题

- 字典树的建树时间复杂度，查询时间复杂度，空间复杂度
    - 构建字典树的时间复杂度: O(n * L)，其中 n 是字符串的数量，L 是字符串的平均长度。
    - 查询字典树的时间复杂度: O(sl)。sl为要查询的字符长长度
    - 构建字典树的空间复杂度: O(n * L * σ)，其中 σ 是字符集的大小。在中文的例子中，字符集 σ 是所有中文字符的集合。如words = ["你好", "世界", "你好世界", "早上好"]，字符集合=["你"、"好"、"世"、"界"、"早"、"上"]

- 条件随机场CRF  
    - CRF 的核心思想是在给定观测序列的条件下，通过最大化条件概率来预测最优的标签序列

    - 解码方式
        - 维特比解码(Viterbi Algorithm)
        - 贪心解码(Greedy Decoding)
        - 束搜素(beam search)
        - 最大边缘解码(Max-Margin Decoding)

- 层级文本分类 Hierarchical text classification (HTC)
    - 层级文本分类和传统文本分类最大区别是将层级label信息融入到文本向量中
    - 一个比较难理解的点：层级label是什么？答是整个数据集的label树状图，将这一整个tree融合到文本向量中
    - 层结构编码广泛应用的方法：Tree-LSTM,GCN
    - 模型训练数据的label为"label": ["C151", "C15", "CCAT"]，虽然是label树的一条路径，但是吧这条路径作为一个label,所以和传统的文本分类模型最后一个一致用sigmoid的做len(label_map)分类。

* 产再摘要生成(保险行业，帮助业务人员判断保险案件是否属实)
    需求：根据保险人记录的“案件名称”、“出险时间”，“事故说明”基础上用附件摘要进行补充，最终生成产再任务的整体摘要，业务人员根据这个摘要来判断该保险案件是否属实。

* RAG(保险咨询应用，医疗诊断协助针对医生的)
    向量数据库
    web搜索
* 数字人(睿医助手，针对患者的，)
    语音识别
    ai智能体
    语音生成
    数字人形象
* text2sql(保险行业，辅助业务人员查询相应数据统计)
    大模型-code,基于大模型进行text2sql任务
    微调：数据生成，设计query问题，根据问题编写相应sql模版,如：[险种]出险次数最多的是哪家公司


In [ ]:
"""
缩放因子，调节点积注意力中的内积大小，防止得分过大或过小，从而防止梯度消失或爆炸问题
点积的值之所以会变得很大，主要是由于向量维度的增加。可以从数学和统计的角度来看点积的计算过程。
点积的期望值和方差：期望值E[qk]=0,方差Var[qk]=dk,
随着维度dk增加，点积的方差dk也会增大。这意味着点积值的绝对值会随着维度的增加而变得越来越大，点积的数值范围会变得非常大。
"""
"""
temperature 和 top_p 的采样原理
不应用该原理的话会导致生成的结构出现重复、单调问题
问题举例：输入: "今天的天气真好，"
输出: "今天的天气真好，所以我决定去散步。今天的天气真好，所以我决定去散步。今天的天气真好，所以我决定去散步。"
操作步骤：第一步，根据top_p和temperature进行tokens的初选，在从初选出来的tokens中看各token的似然分值来进行抽样
似然分值解释：'token1token2token3token4token5',该句话比较成立的概率
"""

In [ ]:
#TOT思维树，实现过程


class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.value = None

    def add_child(self, child_state):
        child_node = Node(child_state, self)
        self.children.append(child_node)
        return child_node

def evaluate_state(state):
    # 评估当前状态的价值，可以根据具体任务进行调整
    # 返回值可以是"sure"、"maybe"或"impossible"
    pass

def expand_node(node):
    # 根据当前节点生成子节点
    new_states = generate_new_states(node.state)  # 假设有一个生成新状态的函数
    for state in new_states:
        child_node = node.add_child(state)
        child_node.value = evaluate_state(state)

def tree_of_thoughts_search(initial_state):
    root = Node(initial_state)
    frontier = [root]
    
    while frontier:
        node = frontier.pop(0)  # 广度优先搜索
        expand_node(node)
        for child in node.children:
            if child.value == "sure":
                return child  # 找到解决方案
            elif child.value == "maybe":
                frontier.append(child)
    return None

# 假设初始状态为initial_state
initial_state = ...  # 定义初始状态
solution_node = tree_of_thoughts_search(initial_state)

if solution_node:
    # 打印解决方案
    solution_path = []
    while solution_node:
        solution_path.append(solution_node.state)
        solution_node = solution_node.parent
    solution_path.reverse()
    print("Solution found:", solution_path)
else:
    print("No solution found.")


```python
    #参数解析
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    #下面三行这样写的方式叫做类型注解，可以做到类型检查
    model_args: ModelArguments
    data_args: DataArguments
    training_args: TrainingArguments
```

# 智源embedding模型训练任务介绍
- bge-m3
    - 模型通过分阶段来训练embedding，第一阶段预训练，第二阶段基于稠密检索的对比学习
    - 数据集格式为{query:string1,pos:[string2],neg:[string3,string4,..]}
    - 模型训练做的任务为：让query-pos的距离更近，query-neg的距离更远，模型会针对query和doc分别进行编码，比较采用了混合检索:稠密检索(Dense retrieval)、词典检索(Lexical retrieval)、多向量召回(Multi-vec retrieval)
    - 模型处理流程：通过基模型roberta进行文本特征提取，采用混合检索计算相似度，和target进行loss计算，总loss也是一种加权混合计算的方式
    - loss求法,code实例:
    ```python
    import torch
    # 假设的输入数据
    q_dense_vecs = torch.randn(3, 768)  # 3个查询向量，每个是768维
    p_dense_vecs = torch.randn(15, 768) # 3个查询，每个对应5个文档（1个正样本+4个负样本），每个文档是768维

    # 生成 targets
    idxs = torch.arange(q_dense_vecs.size(0), device=q_dense_vecs.device, dtype=torch.long)
    targets = idxs * (p_dense_vecs.size(0) // q_dense_vecs.size(0))  # [0, 5, 10]

    # 计算相似度矩阵
    similarity_matrix = torch.matmul(q_dense_vecs, p_dense_vecs.T)  # (3, 15) 每个查询对所有15个文档的相似度

    # 定义损失函数并计算损失
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(similarity_matrix, targets)  # 计算损失

    # 打印损失值
    print(loss)
    ```



- bge-rerank
    - 数据格式和bge-m3的一样，
    - 模型训练任务是判断query-pos是否相似label为0/1，query-neg是否相似label为0/1，基模型为roberta,模型会对query与doc的拼接一起进行编码，分类层做的是sigmoid打分，这里的样本分组有一个处理，每个样本有一个qeury,一个pos，多个neg,每个这样的样本会处理为一组，这样这一组数据会打对应长度的分，label为正样本的位置index

# 文本多分类和多标签，最后分类层设计
- 多文本分类
    - 输出层神经元数量：等于类别总数。激活函数：使用softmax函数(激活函数是内置在损失函数中的)。损失函数：使用交叉熵损失（Cross-Entropy Loss）。
    - label数据格式和损失计算
    ```python
    import torch
    import torch.nn as nn
    
    # 假设模型的输出logits和真实类别索引
    logits = torch.randn(10, 3)  # 假设有10个样本，3个类别
    targets = torch.tensor([0, 2, 1, 1, 0, 2, 0, 2, 1, 0])  # 真实类别索引

    # 定义交叉熵损失函数
    criterion = nn.CrossEntropyLoss()

    # 计算损失
    loss = criterion(logits, targets)
    print("Cross-Entropy Loss:", loss.item())
    ```
- 多标签
    - 输出层神经元数量：等于类别总数。激活函数：使用sigmoid函数(激活函数是内置在损失函数中的)。损失函数：使用二元交叉熵损失（Binary Cross-Entropy Loss）。
    - label数据格式和损失计算
    ```python
    # 假设模型的输出logits和真实标签（one-hot编码）
    logits = torch.randn(10, 3)  # 假设有10个样本，每个样本3个标签
    targets = torch.tensor([[0, 1, 0], [1, 0, 1], [0, 1, 1], ..., [1, 0, 0]])  # 真实标签（one-hot编码）

    # 定义二元交叉熵损失函数
    criterion = nn.BCEWithLogitsLoss()

    # 计算损失
    loss = criterion(logits, targets.float())  # 注意：targets需要是float类型
    print("Binary Cross-Entropy Loss:", loss.item())
    ```
- 多任务
    - 多任务情况下，每个任务会出一个loss